## Importing Libraries and Loading the Data

In [1094]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pickle

In [1095]:
news_df = pd.read_csv("/content/news.csv")
rec_users_df = pd.read_csv("/content/rec_users.csv")

In [1096]:
news_df.head()

,id,title,description,published_date,breaking_news,blob_image,source_url,created_at,updated_at,published_at,created_by_id,updated_by_id,shares,comment_count,type
0,23996,අද ඩොලරයේ අගය,ශ්‍රී ලංකා මහ බැංකුව විසින් අද (13) දින නිකුත්...,2025-02-13 12:10:32.919,False,NaN,https://www.hirunews.lk/396721/%E0%B6%85%E0%B6...,2025-02-13 09:43:07.843,2025-02-13 09:43:12.403,2025-02-13 09:43:12.131,7.0,7.0,0,0,News
1,23995,‘ක්ලීන් ශ‍්‍රී ලංකා ලිඛිතව තියෙනවා.’ මාලිමා මන...,‘ක්‍ලීන් ශ්‍රී ලංකා’ වැඩපිළිවෙල යනු කුමක්දැයි ...,2025-02-13 09:30:00,False,NaN,https://lankacnews.com/%e0%b6%9a%e0%b7%8a%e0%b...,2025-02-13 09:42:37.114,2025-02-13 09:42:47.003,NaN,NaN,NaN,0,0,News
2,23994,පාපන්දු ගෝල කණුවක් කඩා වැටී පාසැල් සිසුවෙකු ජී...,පාසැල් ක්‍රීඩාගංනයක තිබූ පාපන්දු ගෝල කණුවක් කඩ...,2025-02-13 11:10:47.133,False,NaN,https://www.hirunews.lk/396719/%E0%B6%B4%E0%B7...,2025-02-13 09:33:20.185,2025-02-13 09:33:23.796,2025-02-13 09:33:23.747,7.0,7.0,0,0,News
3,23993,සුජීව සේනසිංහගේ මූලික අයිතිවාසිකම් පෙත්සම විභා...,තමන්ට එරෙහිව අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුව විසි...,2025-02-13 10:10:29.598,False,NaN,https://www.hirunews.lk/396716/%E0%B7%83%E0%B7...,2025-02-13 09:31:55.442,2025-02-13 09:31:59.501,2025-02-13 09:31:59.41,7.0,7.0,0,0,News
4,23992,නීති විරෝධී ධීවර දැල් දෙසීය පනහක් නීතියේ රැහැනට,"ශ්‍රී ලංකා නාවික හමුදාව, කිලිනොච්චිය මුද්දලම්ප...",2025-02-13 10:58:56.639,False,NaN,https://www.dinamina.lk/2025/02/13/lawnorder/1...,2025-02-13 09:29:23.368,2025-02-13 09:29:26.299,2025-02-13 09:29:26.258,7.0,7.0,0,0,News


In [1097]:
rec_users_df.head()

,id,user_id,labels,subscribe,comment,created_at,updated_at,created_by_id,updated_by_id
0,99041,2330,[],[],userca4c9...,2025-02-11 11:06:50.796705,2025-02-11 11:06:52.133754,NaN,NaN
1,97587,2329,"[""විදේශීය පුවත්"", ""ගොසිප්"", ""ක්‍රීඩා "", ""දේශීය...",[],Gayani Dharmasuriya,2025-02-10 03:46:13.053964,2025-02-13 05:03:39.931046,NaN,NaN
2,96294,2327,"[""ගොසිප්""]",[],userde5cd...,2025-02-06 08:51:22.097905,2025-02-06 08:57:47.921889,NaN,NaN
3,95623,2326,"[""ගොසිප්""]",[],userdf7a8...,2025-02-05 06:27:46.958448,2025-02-05 06:28:00.833622,NaN,NaN
4,95534,2325,"[""විදේශීය පුවත්"", ""ක්‍රීඩා "", ""දේශීය පුවත්"", ""...",[],user11178...,2025-02-05 05:16:37.895372,2025-02-13 03:43:22.727362,NaN,NaN


In [1098]:
news_df.tail()

,id,title,description,published_date,breaking_news,blob_image,source_url,created_at,updated_at,published_at,created_by_id,updated_by_id,shares,comment_count,type
995,22983,එමිල් රංජන්ට දුන් සම්මානය ගැන ඇමති නලින්දගෙන් ...,රජයේ භාරයේ සිටින පුරවැසියන් ඝාතනය කිරීම කිසි ස...,2025-01-28 09:45:00,False,NaN,https://sinhala.adaderana.lk/news/205935,2025-01-28 09:56:04.219,2025-01-28 10:01:07.455,2025-01-28 10:01:07.354,NaN,1.0,0,0,News
996,22982,(Video) ජීවිතය හරියට plan කරපු මිනිස්සුන්ගේ පො...,අද මේ වීඩියෝ එකේ මම කතා කරන්න යන්නේ ෆයර් (FIRE...,2025-01-28 11:20:03.74,False,NaN,https://www.youtube.com/watch?v=NC7jXIxqR38,2025-01-28 09:55:08.167,2025-01-28 09:55:13.987,2025-01-28 09:55:13.936,1.0,1.0,0,0,Video
997,22981,(Video) 5000 නෝට්ටු අවලංගු කළොත්? ආණ්ඩුව හිරවි...,- Lanka Business TV,2025-01-28 09:51:25.823062,False,NaN,https://www.youtube.com/watch?v=hOnKPBi8HJw,2025-01-28 09:51:25.81,2025-01-28 09:51:31.391,2025-01-28 09:51:31.31,1.0,1.0,0,0,Video
998,22980,කැළණියේ නයා ගැන මට කීවේ මාධ්‍යවේදියෙක් - දිලිත්,කැලණි රජමහ විහාරයෙන් නාග ධාතු මතු වීමේ සිද්ධිය...,2025-01-28 09:30:00,False,NaN,https://lankacnews.com/%e0%b6%9a%e0%b7%90%e0%b...,2025-01-28 09:42:33.071,2025-01-28 09:47:19.634,2025-01-28 09:47:19.565,NaN,1.0,0,0,News
999,22979,අංජනමක් බලා නිධන් හොයන්න නිවස මැද අඩි 18 ක් හාරලා,අංජනම් බලන්නෙකුගේ උපදෙස් මත නිවසක රත්රන් නිධාන...,2025-01-28 13:35:55.315,False,NaN,https://www.gossiplankanews.com/2025/01/they-d...,2025-01-28 09:19:29.835,2025-01-28 09:19:33.207,2025-01-28 09:19:33.117,7.0,7.0,0,0,News


In [1099]:
rec_users_df.tail()

,id,user_id,labels,subscribe,comment,created_at,updated_at,created_by_id,updated_by_id
995,13617,1228,"[""විදේශීය පුවත්"", ""ගොසිප්"", ""ක්‍රීඩා "", ""දේශීය...",[],user6cb4c...,2024-11-16 13:10:25.751283,2025-01-19 16:20:37.234682,NaN,NaN
996,13612,1227,"[""දේශීය පුවත්""]",[],user2c21b...,2024-11-16 13:07:24.898794,2024-11-17 05:05:59.198047,NaN,NaN
997,13607,1226,"[""ගොසිප්""]",[],usere73ca...,2024-11-16 13:06:03.332952,2024-11-20 03:04:18.494742,NaN,NaN
998,13600,1225,"[""ගොසිප්""]",[],user41de7...,2024-11-16 13:04:32.37607,2024-11-28 08:13:38.554099,NaN,NaN
999,13590,1224,"[""දේශපාලන""]",[],userd4ef6...,2024-11-16 13:02:32.967802,2025-01-08 05:43:05.026995,NaN,NaN


In [1100]:
news_df.describe()

,id,blob_image,created_by_id,updated_by_id,shares,comment_count
count,1000.000000,0.0,618.000000,965.000000,1000.000000,1000.000000
mean,23489.186000,NaN,6.077670,4.376166,0.022000,0.004000
std,293.105265,NaN,2.165844,2.977866,0.222633,0.077395
min,22979.000000,NaN,1.000000,1.000000,0.000000,0.000000
25%,23238.750000,NaN,7.000000,1.000000,0.000000,0.000000
50%,23488.500000,NaN,7.000000,7.000000,0.000000,0.000000
75%,23742.250000,NaN,7.000000,7.000000,0.000000,0.000000
max,23996.000000,NaN,7.000000,7.000000,4.000000,2.000000


In [1101]:
rec_users_df.describe()

,id,user_id,created_by_id,updated_by_id
count,1000.000000,1000.000000,0.0,0.0
mean,34309.823000,1756.014000,NaN,NaN
std,19403.635631,341.583461,NaN,NaN
min,13590.000000,179.000000,NaN,NaN
25%,18288.750000,1480.750000,NaN,NaN
50%,29660.500000,1763.500000,NaN,NaN
75%,43363.000000,2039.250000,NaN,NaN
max,99041.000000,2330.000000,NaN,NaN


In [1102]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1000 non-null   int64  
 1   title           1000 non-null   object 
 2   description     1000 non-null   object 
 3   published_date  1000 non-null   object 
 4   breaking_news   1000 non-null   bool   
 5   blob_image      0 non-null      float64
 6   source_url      1000 non-null   object 
 7   created_at      1000 non-null   object 
 8   updated_at      1000 non-null   object 
 9   published_at    965 non-null    object 
 10  created_by_id   618 non-null    float64
 11  updated_by_id   965 non-null    float64
 12  shares          1000 non-null   int64  
 13  comment_count   1000 non-null   int64  
 14  type            1000 non-null   object 
dtypes: bool(1), float64(3), int64(3), object(8)
memory usage: 110.5+ KB


In [1103]:
rec_users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   int64  
 1   user_id        1000 non-null   int64  
 2   labels         1000 non-null   object 
 3   subscribe      1000 non-null   object 
 4   comment        989 non-null    object 
 5   created_at     989 non-null    object 
 6   updated_at     989 non-null    object 
 7   created_by_id  0 non-null      float64
 8   updated_by_id  0 non-null      float64
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


In [1104]:
news_df.dtypes

,0
id,int64
title,object
description,object
published_date,object
breaking_news,bool
blob_image,float64
source_url,object
created_at,object
updated_at,object
published_at,object


In [1105]:
rec_users_df.dtypes

,0
id,int64
user_id,int64
labels,object
subscribe,object
comment,object
created_at,object
updated_at,object
created_by_id,float64
updated_by_id,float64


In [1106]:
news_df.shape

(1000, 15)

In [1107]:
rec_users_df.shape

(1000, 9)

In [1108]:
news_df.columns

Index(['id', 'title', 'description', 'published_date', 'breaking_news',
       'blob_image', 'source_url', 'created_at', 'updated_at', 'published_at',
       'created_by_id', 'updated_by_id', 'shares', 'comment_count', 'type'],
      dtype='object')

In [1109]:
rec_users_df.columns

Index(['id', 'user_id', 'labels', 'subscribe', 'comment', 'created_at',
       'updated_at', 'created_by_id', 'updated_by_id'],
      dtype='object')

## Data Preprocessing

In [1110]:
news_df.isnull().sum()

,0
id,0
title,0
description,0
published_date,0
breaking_news,0
blob_image,1000
source_url,0
created_at,0
updated_at,0
published_at,35


In [1111]:
rec_users_df.isnull().sum()

,0
id,0
user_id,0
labels,0
subscribe,0
comment,11
created_at,11
updated_at,11
created_by_id,1000
updated_by_id,1000


In [1112]:
# Handle missing values in news dataset
news_df['blob_image'] = news_df['blob_image'].fillna('')  # Fill missing image URLs with empty string
news_df['published_at'] = news_df['published_at'].fillna(pd.NaT)  # Fill with NaT (Not a Time)
news_df['created_by_id'] = news_df['created_by_id'].fillna(0)  # Fill missing IDs with 0
news_df['updated_by_id'] = news_df['updated_by_id'].fillna(0)  # Fill missing IDs with 0

print(news_df.head())

      id                                              title  \
0  23996                                      අද ඩොලරයේ අගය   
1  23995  ‘ක්ලීන් ශ‍්‍රී ලංකා ලිඛිතව තියෙනවා.’ මාලිමා මන...   
2  23994  පාපන්දු ගෝල කණුවක් කඩා වැටී පාසැල් සිසුවෙකු ජී...   
3  23993  සුජීව සේනසිංහගේ මූලික අයිතිවාසිකම් පෙත්සම විභා...   
4  23992    නීති විරෝධී ධීවර දැල් දෙසීය පනහක් නීතියේ රැහැනට   

                                         description           published_date  \
0  ශ්‍රී ලංකා මහ බැංකුව විසින් අද (13) දින නිකුත්...  2025-02-13 12:10:32.919   
1  ‘ක්‍ලීන් ශ්‍රී ලංකා’ වැඩපිළිවෙල යනු කුමක්දැයි ...      2025-02-13 09:30:00   
2  පාසැල් ක්‍රීඩාගංනයක තිබූ පාපන්දු ගෝල කණුවක් කඩ...  2025-02-13 11:10:47.133   
3  තමන්ට එරෙහිව අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුව විසි...  2025-02-13 10:10:29.598   
4  ශ්‍රී ලංකා නාවික හමුදාව, කිලිනොච්චිය මුද්දලම්ප...  2025-02-13 10:58:56.639   

   breaking_news blob_image  \
0          False              
1          False              
2          False              
3         

In [1113]:
news_df = news_df.drop(columns=['blob_image','shares', 'comment_count', 'type'])

In [1114]:
news_df.columns

Index(['id', 'title', 'description', 'published_date', 'breaking_news',
       'source_url', 'created_at', 'updated_at', 'published_at',
       'created_by_id', 'updated_by_id'],
      dtype='object')

In [1115]:
# Handle missing values in rec_users dataset
rec_users_df['comment'] = rec_users_df['comment'].fillna('')  # Fill with empty string if comments are missing
rec_users_df['created_at'] = rec_users_df['created_at'].fillna(pd.NaT)  # Fill with NaT (Not a Time)
rec_users_df['updated_at'] = rec_users_df['updated_at'].fillna(pd.NaT)  # Fill with NaT (Not a Time)
rec_users_df['created_by_id'] = rec_users_df['created_by_id'].fillna(0)  # Fill with 0 if ID is missing
rec_users_df['updated_by_id'] = rec_users_df['updated_by_id'].fillna(0)  # Fill with 0 if ID is missing

print(rec_users_df.head())

      id  user_id                                             labels  \
0  99041     2330                                                 []   
1  97587     2329  ["විදේශීය පුවත්", "ගොසිප්", "ක්‍රීඩා ", "දේශීය...   
2  96294     2327                                         ["ගොසිප්"]   
3  95623     2326                                         ["ගොසිප්"]   
4  95534     2325  ["විදේශීය පුවත්", "ක්‍රීඩා ", "දේශීය පුවත්", "...   

  subscribe              comment                  created_at  \
0        []         userca4c9...  2025-02-11 11:06:50.796705   
1        []  Gayani Dharmasuriya  2025-02-10 03:46:13.053964   
2        []         userde5cd...  2025-02-06 08:51:22.097905   
3        []         userdf7a8...  2025-02-05 06:27:46.958448   
4        []         user11178...  2025-02-05 05:16:37.895372   

                   updated_at  created_by_id  updated_by_id  
0  2025-02-11 11:06:52.133754            0.0            0.0  
1  2025-02-13 05:03:39.931046            0.0            0.

In [1116]:
rec_users_df = rec_users_df.drop(columns=['subscribe', 'comment', 'created_at', 'updated_at', 'created_by_id', 'updated_by_id'])

In [1117]:
rec_users_df.columns

Index(['id', 'user_id', 'labels'], dtype='object')

In [1118]:
# Create a 'content' column for TF-IDF (combining 'title' and 'description')
news_df['content'] = news_df['title'] + " " + news_df['description']

In [1119]:
# Reset the index after data preprocessing
news_df.reset_index(drop=True, inplace=True)

In [1120]:
# Split the dataset into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(news_df, test_size=0.2, random_state=42)

In [1121]:
# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

In [1122]:
# Fit and transform the content of news articles
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_df['content'])

In [1123]:
# Calculate cosine similarity between all training articles
cosine_sim_train = cosine_similarity(tfidf_matrix_train, tfidf_matrix_train)

In [1124]:
# Function to recommend top N similar articles (based on training data)
def recommend_news(article_id, top_n=5, cosine_sim=cosine_sim_train):
    # Get pairwise similarity scores for the given article
    sim_scores = list(enumerate(cosine_sim[article_id]))

    # Sort the articles based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top N similar articles
    similar_indices = [i[0] for i in sim_scores[1:top_n+1]]  # Exclude the article itself

    # Return the recommended articles (excluding the original article)
    return train_df.iloc[similar_indices]

In [1125]:
# Save the TF-IDF Vectorizer and model
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

# Save the cosine similarity matrix (for training set)
np.save('cosine_similarity_train.npy', cosine_sim_train)

In [1127]:
# Test the recommendation with loaded models
recommended_articles = recommend_news(article_id=0, top_n=10)
print(recommended_articles[['title']])

                                                 title
32                  රජයේ මුද‍්‍රණාලයේ උණුසුම් තත්වයක්.
886  මාස අටක වැටුප අරන් ඉල්ලා අස්වෙන්න.ට්‍රම්ප් මිල...
219                        ගුවන්තොටෙන් හමුවූ ජීව උණ්ඩය
894          ආරක්ෂක මාණ්ඩලික ප්‍රධානී තනතුර අහෝසි කරයි
58   ඉන්දු ශ්‍රී ලංකා ආරක්‍ෂක ලේකම්වරු අතර හමුවක් ඉ...
333  UN මෙරට නේවාසික සම්බන්ධීකාරක ආරක්ෂක ලේකම් හමුවෙයි
847                           යෝෂිත ළඟ ගිනි අවි හතක් ?
108                                පොලිසියට නව තනතුරක්
900  ‘ආරක්ෂක මාණ්ඩලික ප්‍රධානී’ තනතුර දැනට අවශ්‍ය න...
240  උසාවි නියෝගයෙන් ගෙයි අයිතිය ගන්න ගිහාම යන්න බෑ...
